<a href="https://colab.research.google.com/github/rushweigelt/Machine_Learning_Twitter_Analysis_Tool/blob/master/openai_antiracistbot/antiracist_try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#mount personal google drive to access files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Make a few directories for later, then most importantly ocme back to content for permission reasons of all thoings
#%cd drive
#%cd My\ Drive
#%mkdir bot
%cd /content/
!ls

In [0]:
#%cd gpt-2

In [0]:
#Clone into /content
!git clone https://github.com/mohamad-ali-nasser/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 395, done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 395
Receiving objects: 100% (395/395), 4.43 MiB | 6.76 MiB/s, done.
Resolving deltas: 100% (216/216), done.


In [7]:
#grab medium
%cd gpt-2
!python3 download_model.py 345M

/content/gpt-2
Fetching checkpoint: 1.00kit [00:00, 743kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 57.1Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 780kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:20, 69.6Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 5.18Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 47.2Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 38.1Mit/s]                                                       


In [0]:
#too large for free collab notbooks :(
#!python3 download_model.py 774M

In [0]:
#load old checkpoints
!cp -r /content/drive/My\ Drive/bot/checkpoint/run1/* /content/gpt-2/models/345M

In [0]:
#load previous checkpoint
#!cp -r /content/drive/My\ Drive/bot/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M

In [0]:
#make folder to drop data into
%cd src
%mkdir sample_data
%cd sample_data
!export PYTHONIOENCODING=UTF-8

In [0]:
#go back to outer gtp-2 and then install reqs
%cd ..
%cd ..
!pip3 install -r requirements.txt

In [0]:
#Get right version of tf
!pip install tensorflow-gpu==1.15.0
!pip install 'tensorflow-estimator<1.15.0rc0,>=1.14.0rc0' --force-reinstall

In [0]:
#Insane ass cuda stuff, bleh, cuda needs to be such an exact version
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-*/7fa2af80.pub
!apt-get update
!apt-get install cuda-9-0
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.0/lib64/

In [0]:
#%cd gpt-2
#!ls

In [0]:
#where we train, run from gpt-2
!PYTHONPATH=src ./train.py --dataset src/sample_data/s.txt --model_name '345M'

In [0]:
#copy checkpoint to personal drive and the model
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/bot/checkpoint
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Now that model has been updated, lets interact



In [0]:
#skip cell generally
import os
%cd src
from conditional_model import conditional_model
%cd ..

In [0]:
#skip cell
conditional_model(seed=1,sentences=['How are you today?', 'Hi i am here'])

In [0]:
#start back here
%cd src
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [0]:
%cd ..

In [0]:
#needs to be run from gpt-2 directory
interact_model(
    '345M',
    None,
    1,
    1,
    100,
    1,
    0,
    '/content/gpt-2/models'
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> What is white nationalism?
======================================== SAMPLE 1 ========================================


White nationalism says that people should be united and hate getting disapproved too much by others: they should feel they are equal because they don't want to get punished or annoyed. When people influence your life, everyone will be affected; if there is someone who annoys you, you can't just stay silent about it etc etc. Therefore, in the white politician presence it is dangerous because it requires a protest and takes many people out of everyday life.

What is the state of the
Model prompt >>> Why is white nationalism so common in the United States?
======================================== SAMPLE 1 ========================================


It's morally